# Download the .nc files and save in a seperate folder.

In [40]:
import os
import requests

def download_files_from_links(filename):
    folder_name = os.path.splitext(filename)[0]  # Extract the folder name from the text file name
    os.makedirs(folder_name, exist_ok=True)  # Create the folder if it doesn't exist

    with open(filename, 'r') as file:
        links = file.readlines()

    for link in links:
        link = link.strip()
        file_name = link.split('/')[-1].split('?')[0]  # Extract the file name from the URL
        file_path = os.path.join(folder_name, file_name)  # Create the file path within the folder

        print(f"Downloading {file_name}...")

        response = requests.get(link)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                file.write(response.content)
            print(f"{file_name} downloaded successfully.")
        else:
            print(f"Failed to download {file_name}.")

# Provide the filename of the text file containing the links
filename = "/Users/jennifergao/ClimateData_NT/S202306141138084833207.txt"
download_files_from_links(filename)


Z_NAFP_C_BABJ_20201101000616_P_CLDAS_NRT_ASI_0P0625_DAY-SSRA-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000614_P_CLDAS_NRT_ASI_0P0625_DAY-PRE-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000616_P_CLDAS_NRT_ASI_0P0625_DAY-WIN-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000615_P_CLDAS_NRT_ASI_0P0625_DAY-PRS-2020102900.nc downloaded successfully.
Ic1V9R8S6OqV3h9sbNG3FUI%3D&dataCode=NAFP_CLDAS2.0_NRT&userId=3427D0BD10D4786B96871F0E9F8FE6BD downloaded successfully.
Z_NAFP_C_BABJ_20201101000616_P_CLDAS_NRT_ASI_0P0625_DAY-SHU-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000823_P_CLDAS_NRT_ASI_0P0625_DAY-GST-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000830_P_CLDAS_NRT_CHN_0P0625_DAY-RSM000010-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000841_P_CLDAS_NRT_ASI_0P0625_DAY-SM010040-2020102900.nc downloaded successfully.
Z_NAFP_C_BABJ_20201101000831_P_CLDAS_NRT_CHN_0P0625_DAY-RSM000050-2020102900.

# read the nc file

In [46]:
import os
import netCDF4 as nc
import openpyxl as xl
import numpy as np

# 设置文件夹路径
folder_path = '/Users/jennifergao/ClimateData_NT/S202306141138084833206'

# Get all nc files in the folder
file_list = [file for file in os.listdir(folder_path) if file.endswith('.nc')]

# Create an empty list to store the data
data_list = []

# Create an Excel file
excel_file = '1003-1005.xlsx'
if os.path.exists(excel_file):
    os.remove(excel_file)

# Loop through each nc file
for file_name in file_list:
    # Read the nc file
    file_path = os.path.join(folder_path, file_name)
    nc_info = nc.Dataset(file_path)
    variable_keys = list(nc_info.variables.keys())
    var_data = np.array(nc_info.variables[variable_keys[-1]][:])  # Convert the data to a numpy array
    var_data_flattened = var_data.flatten().tolist()  # Flatten the data and convert to a list
    data_list.append([file_name, variable_keys[-1], var_data_flattened])
    nc_info.close()

# Specify the filename and sheet name for the Excel file
filename = '1003-1005.xlsx'
sheetname = 'Sheet1'

# Write the data to the Excel file
workbook = xl.Workbook()
sheet = workbook.active
for row in data_list:
    cleaned_row = []
    for value in row:
        if isinstance(value, list):
            # Flatten the nested list and replace large values with a placeholder
            cleaned_sublist = [sub_value if (isinstance(sub_value, str) or abs(sub_value) < 1e35) else "N/A" for sub_value in value]
            cleaned_row.extend(cleaned_sublist)
        else:
            # Replace large values with a placeholder
            cleaned_value = value if (isinstance(value, str) or abs(value) < 1e35) else "N/A"
            cleaned_row.append(cleaned_value)
    sheet.append(cleaned_row)
workbook.save(filename)


# Combine the data

In [47]:
import openpyxl as xl

# Set the Excel filename and data start row
filename = '1003-1005.xlsx'
data_start_row = 1

# Load the data from the Excel file
workbook = xl.load_workbook(filename)
sheet = workbook.active
raw_data = list(sheet.iter_rows(values_only=True))[data_start_row-1:]

# Process each row of data
variable_meaning = []
data = []
for row in raw_data:
    file_name, variable_name, variable_value = row[:3]
    
    # Add a check here
    if not isinstance(file_name, str):
        print(f"Unexpected data type for file_name: {file_name} ({type(file_name)})")
        continue

    datetime = file_name[14:23]  # Extract the datetime string from the file name


    # Check the variable name and carry out different operations depending on its value
    if variable_name == 'PRCP_DAY_SUM':
        variable_meaning.append('1小时相对降水')
    elif variable_name == 'PAIR_DAY_AVG':
        variable_meaning.append('地面气压')
    elif variable_name == 'QAIR_DAY_AVG':
        variable_meaning.append('比湿')
    elif variable_name == 'SWDN_DAY_AVG':
        variable_meaning.append('短波辐射')
    elif variable_name == 'TAIR_DAY_AVG':
        variable_meaning.append('2m气温')
    elif variable_name == 'WIND_DAY_AVG':
        variable_meaning.append('10m风速')
    elif variable_name == 'TG_DAY_MIN':
        variable_meaning.append('地表温度')
    elif variable_name == 'SOILLIQ':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'SM000005':
            variable_meaning.append('0-5cm土壤湿度')
        elif var_name == 'SM000010':
            variable_meaning.append('0-10cm土壤湿度')
        elif var_name == 'SM010040':
            variable_meaning.append('10-40cm土壤湿度')
        elif var_name == 'SM040100':
            variable_meaning.append('40-100cm土壤湿度')
        elif var_name == 'SM100200':
            variable_meaning.append('100-200cm土壤湿度')
        else:
            variable_meaning.append('')
    elif variable_name == 'TSOI':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'GST005':
            variable_meaning.append('5cm土壤温度')
        elif var_name == 'GST010':
            variable_meaning.append('10cm土壤温度')
        elif var_name == 'GST040':
            variable_meaning.append('40cm土壤温度')
        elif var_name == 'GST100':
            variable_meaning.append('100cm土壤温度')
        elif var_name == 'GST200':
            variable_meaning.append('200cm土壤温度')
    elif variable_name == 'RSM':
        parts = file_name.split('-')
        var_name = parts[-2]
        if var_name == 'RSM000010':
            variable_meaning.append('0-10cm土壤相对湿度')
        elif var_name == 'RSM000020':
            variable_meaning.append('0-20cm土壤相对湿度')
        elif var_name == 'RSM000050':
            variable_meaning.append('0-50cm土壤相对湿度')
        else:
            variable_meaning.append('')
    else:
        variable_meaning.append('')

    # Add the file name, datetime, variable name, variable value and variable meaning to the data list
    data.append([file_name, datetime, variable_name, variable_value, variable_meaning[-1]])

# Write the data to the Excel file
output_sheet = workbook.create_sheet(title='Sheet2')
output_sheet.append(['File Name', 'Datetime', 'Variable Name', 'Variable Value', 'Variable Meaning'])
for row in data:
    output_sheet.append(row)

# Save the Excel file
workbook.save(filename)

# Display the results
print('Data with variable meanings:')
for row in data:
    print(row)


Data with variable meanings:
['Z_NAFP_C_BABJ_20201030000631_P_CLDAS_NRT_ASI_0P0625_DAY-SSRA-2020102700.nc', '202010300', 'SWDN_DAY_AVG', 66.34351348876953, '短波辐射']
['Z_NAFP_C_BABJ_20201101000831_P_CLDAS_NRT_CHN_0P0625_DAY-RSM000050-2020102900.nc', '202011010', 'RSM', 'N/A', '0-50cm土壤相对湿度']
['Z_NAFP_C_BABJ_20201030000848_P_CLDAS_NRT_ASI_0P0625_DAY-GST200-2020102700.nc', '202010300', 'TSOI', 'N/A', '200cm土壤温度']
['Z_NAFP_C_BABJ_20201031000626_P_CLDAS_NRT_ASI_0P0625_DAY-PRS-2020102800.nc', '202010310', 'PAIR_DAY_AVG', 102507.96875, '地面气压']
['Z_NAFP_C_BABJ_20201031000841_P_CLDAS_NRT_ASI_0P0625_DAY-SM040100-2020102800.nc', '202010310', 'SOILLIQ', 'N/A', '40-100cm土壤湿度']
['Z_NAFP_C_BABJ_20201101000846_P_CLDAS_NRT_ASI_0P0625_DAY-GST040-2020102900.nc', '202011010', 'TSOI', 'N/A', '40cm土壤温度']
['Z_NAFP_C_BABJ_20201030000844_P_CLDAS_NRT_ASI_0P0625_DAY-SM000005-2020102700.nc', '202010300', 'SOILLIQ', 'N/A', '0-5cm土壤湿度']
['Z_NAFP_C_BABJ_20201101000847_P_CLDAS_NRT_ASI_0P0625_DAY-GST100-2020102900.nc',